This note booke is mainly from  https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution

I do not own anything

In [1]:
# import libraries 
import numpy as np
import pandas as pd
import seaborn as sns
import gc

# plotting libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# Ignore useless warnings 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
# Reading data

# train test data
df_train = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\application_train.csv")
df_test = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\application_test.csv")

# other data (check kaggle for explanation)
bureau = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\bureau.csv")
bureau_balance = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\bureau_balance.csv")
credit_card_balance = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\credit_card_balance.csv")
installments_payments = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\installments_payments.csv")
POS_CASH_balance = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\POS_CASH_balance.csv")
previous_application = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\previous_application.csv")



# Application 

In [18]:
del bureau
del bureau_balance
del credit_card_balance
del installments_payments
del POS_CASH_balance
del previous_application

In [ ]:
df = pd.concat([df_train, df_test], axis = 0, ignore_index = True)

In [ ]:
# Aggregated features based on External sources
# basically the min, max, mean, median ignoring nan values, variance
# https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution

df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1 * 2 + df.EXT_SOURCE_2 * 1 + df.EXT_SOURCE_3 * 3
np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
for function_name in ['min', 'max', 'mean', 'nanmedian', 'var']:
    feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
    df[feature_name] = eval('np.{}'.format(function_name))(
        df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']], axis=1)


In [ ]:
# more combinations on external sources

df['app EXT_SOURCE_1 * EXT_SOURCE_2'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2']
df['app EXT_SOURCE_1 * EXT_SOURCE_3'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']
df['app EXT_SOURCE_2 * EXT_SOURCE_3'] = df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
df['app EXT_SOURCE_1 * DAYS_EMPLOYED'] = df['EXT_SOURCE_1'] * df['DAYS_EMPLOYED']
df['app EXT_SOURCE_2 * DAYS_EMPLOYED'] = df['EXT_SOURCE_2'] * df['DAYS_EMPLOYED']
df['app EXT_SOURCE_3 * DAYS_EMPLOYED'] = df['EXT_SOURCE_3'] * df['DAYS_EMPLOYED']
df['app EXT_SOURCE_1 / DAYS_BIRTH'] = df['EXT_SOURCE_1'] / df['DAYS_BIRTH']
df['app EXT_SOURCE_2 / DAYS_BIRTH'] = df['EXT_SOURCE_2'] / df['DAYS_BIRTH']
df['app EXT_SOURCE_3 / DAYS_BIRTH'] = df['EXT_SOURCE_3'] / df['DAYS_BIRTH']

In [ ]:
df.shape

# Bureau 

In [ ]:
# feature engineering
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']


In [ ]:
# Replace\remove some outliers in bureau set
bureau.loc[bureau['AMT_ANNUITY'] > .8e8, 'AMT_ANNUITY'] = np.nan
bureau.loc[bureau['AMT_CREDIT_SUM'] > 3e8, 'AMT_CREDIT_SUM'] = np.nan
bureau.loc[bureau['AMT_CREDIT_SUM_DEBT'] > 1e8, 'AMT_CREDIT_SUM_DEBT'] = np.nan
bureau.loc[bureau['AMT_CREDIT_MAX_OVERDUE'] > .8e8, 'AMT_CREDIT_MAX_OVERDUE'] = np.nan
bureau.loc[bureau['DAYS_ENDDATE_FACT'] < -10000, 'DAYS_ENDDATE_FACT'] = np.nan
bureau.loc[(bureau['DAYS_CREDIT_UPDATE'] > 0) | (bureau['DAYS_CREDIT_UPDATE'] < -40000), 'DAYS_CREDIT_UPDATE'] = np.nan
bureau.loc[bureau['DAYS_CREDIT_ENDDATE'] < -10000, 'DAYS_CREDIT_ENDDATE'] = np.nan
bureau.drop(bureau[bureau['DAYS_ENDDATE_FACT'] < bureau['DAYS_CREDIT']].index, inplace = True)

In [ ]:
# Some new features in bureau set
bureau['bureau AMT_CREDIT_SUM - AMT_CREDIT_SUM_DEBT'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['bureau AMT_CREDIT_SUM - AMT_CREDIT_SUM_LIMIT'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_LIMIT']
bureau['bureau AMT_CREDIT_SUM - AMT_CREDIT_SUM_OVERDUE'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_OVERDUE']
bureau['bureau DAYS_CREDIT - CREDIT_DAY_OVERDUE'] = bureau['DAYS_CREDIT'] - bureau['CREDIT_DAY_OVERDUE']
bureau['bureau DAYS_CREDIT - DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_ENDDATE']
bureau['bureau DAYS_CREDIT - DAYS_ENDDATE_FACT'] = bureau['DAYS_CREDIT'] - bureau['DAYS_ENDDATE_FACT']
bureau['bureau DAYS_CREDIT_ENDDATE - DAYS_ENDDATE_FACT'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
bureau['bureau DAYS_CREDIT_UPDATE - DAYS_CREDIT_ENDDATE'] = bureau['DAYS_CREDIT_UPDATE'] - bureau['DAYS_CREDIT_ENDDATE']

In [ ]:
# one hot encoder helper function
def one_hot_encoder(data, nan_as_category = True):
    original_columns = list(data.columns)
    categorical_columns = [col for col in data.columns \
                           if not pd.api.types.is_numeric_dtype(data[col].dtype)]
    for c in categorical_columns:
        if nan_as_category:
            data[c].fillna('NaN', inplace = True)
        values = list(data[c].unique())
        for v in values:
            data[str(c) + '_' + str(v)] = (data[c] == v).astype(np.uint8)
    data.drop(categorical_columns, axis = 1, inplace = True)
    return data, [c for c in data.columns if c not in original_columns]

In [ ]:
bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category=True)

# Bureau Balance

In [ ]:
bureau_balance.head(10)

In [ ]:
bureau_balance, bb_cat = one_hot_encoder(bureau_balance, nan_as_category=True)

In [ ]:
bureau_balance.shape

In [ ]:
bb_agg = bureau_balance.groupby('SK_ID_BUREAU')[bb_cat].mean().reset_index()

In [ ]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()

In [ ]:
agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
bb_processed= group_and_merge(bureau_balance, bb_agg, '', agg, 'SK_ID_BUREAU')
del bureau_balance
gc.collect()

In [ ]:
bb_processed.shape

In [ ]:
# group and merge bureau and bb_processed

bureau = bureau.merge(bb_processed, how='left', on='SK_ID_BUREAU')

In [ ]:
bureau.shape

In [ ]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [ ]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_new.head(10)

In [ ]:
bureau_agg_new.shape

In [ ]:
# group and merge df and bureau_agg_new

df = df.merge(bureau_agg_new, how='left', on='SK_ID_CURR')

In [ ]:
del bureau_agg_new
del bureau
del bb_processed
gc.collect()

In [23]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

Training Data Shape:  (307511, 557)
Testing Data Shape:  (48744, 557)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.760087	train's binary_logloss: 0.252083	valid's auc: 0.74377	valid's binary_logloss: 0.253346
[400]	train's auc: 0.778154	train's binary_logloss: 0.243175	valid's auc: 0.751674	valid's binary_logloss: 0.247543
[600]	train's auc: 0.792108	train's binary_logloss: 0.237705	valid's auc: 0.757617	valid's binary_logloss: 0.24494
[800]	train's auc: 0.803764	train's binary_logloss: 0.233413	valid's auc: 0.761735	valid's binary_logloss: 0.24343
[1000]	train's auc: 0.813627	train's binary_logloss: 0.229813	valid's auc: 0.764563	valid's binary_logloss: 0.242453
[1200]	train's auc: 0.8227	train's binary_logloss: 0.226575	valid's auc: 0.766509	valid's binary_logloss: 0.24181
[1400]	train's auc: 0.831117	train's binary_logloss: 0.223584	v

[1400]	train's auc: 0.830488	train's binary_logloss: 0.223677	valid's auc: 0.769743	valid's binary_logloss: 0.241608
[1600]	train's auc: 0.838077	train's binary_logloss: 0.220933	valid's auc: 0.770464	valid's binary_logloss: 0.241334
[1800]	train's auc: 0.845568	train's binary_logloss: 0.218257	valid's auc: 0.77111	valid's binary_logloss: 0.241116
[2000]	train's auc: 0.852457	train's binary_logloss: 0.215733	valid's auc: 0.771563	valid's binary_logloss: 0.240951
[2200]	train's auc: 0.859032	train's binary_logloss: 0.213332	valid's auc: 0.771887	valid's binary_logloss: 0.240845
[2400]	train's auc: 0.865222	train's binary_logloss: 0.210993	valid's auc: 0.772148	valid's binary_logloss: 0.24075
Early stopping, best iteration is:
[2437]	train's auc: 0.866325	train's binary_logloss: 0.210571	valid's auc: 0.772216	valid's binary_logloss: 0.240729
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores do

In [24]:
metrics_raw

,fold,train,valid
0,0,0.896465,0.772087
1,1,0.901381,0.775254
2,2,0.891585,0.766968
3,3,0.866325,0.772216
4,4,0.866921,0.773372
5,overall,0.884536,0.771933


In [25]:
fi_raw.sort_values(by=['importance'], inplace=True)
drop = list(fi_raw['feature'][1:200])

In [26]:
for i in drop:
    try:
        df.drop(i, inplace=True, axis=1)
    except KeyError:
        continue

In [ ]:
df.shape

In [122]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

Training Data Shape:  (307511, 422)
Testing Data Shape:  (48744, 422)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.760458	train's binary_logloss: 0.252111	valid's auc: 0.744092	valid's binary_logloss: 0.253356
[400]	train's auc: 0.778297	train's binary_logloss: 0.243213	valid's auc: 0.751633	valid's binary_logloss: 0.247579
[600]	train's auc: 0.791821	train's binary_logloss: 0.23779	valid's auc: 0.75738	valid's binary_logloss: 0.245008
[800]	train's auc: 0.803507	train's binary_logloss: 0.23349	valid's auc: 0.761635	valid's binary_logloss: 0.24345
[1000]	train's auc: 0.813562	train's binary_logloss: 0.229866	valid's auc: 0.764187	valid's binary_logloss: 0.242544
[1200]	train's auc: 0.822658	train's binary_logloss: 0.226621	valid's auc: 0.766143	valid's binary_logloss: 0.241887
[1400]	train's auc: 0.831099	train's binary_logloss: 0.223601

[2400]	train's auc: 0.865636	train's binary_logloss: 0.210844	valid's auc: 0.772727	valid's binary_logloss: 0.240579
[2600]	train's auc: 0.871545	train's binary_logloss: 0.208582	valid's auc: 0.772974	valid's binary_logloss: 0.240499
[2800]	train's auc: 0.877075	train's binary_logloss: 0.206352	valid's auc: 0.773142	valid's binary_logloss: 0.240448
[3000]	train's auc: 0.88241	train's binary_logloss: 0.204214	valid's auc: 0.77324	valid's binary_logloss: 0.240408
[3200]	train's auc: 0.887665	train's binary_logloss: 0.202068	valid's auc: 0.773281	valid's binary_logloss: 0.240382
Early stopping, best iteration is:
[3168]	train's auc: 0.886854	train's binary_logloss: 0.202403	valid's auc: 0.773297	valid's binary_logloss: 0.240379
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.759457	train's binary_logloss: 0.252424	valid's auc: 0.74939	valid's 

In [123]:
metrics_raw

,fold,train,valid
0,0,0.892866,0.771673
1,1,0.888115,0.774860
2,2,0.887486,0.767270
3,3,0.886854,0.773297
4,4,0.883404,0.774530
5,overall,0.887745,0.772314


# Previous applications

In [28]:
# Replace some outliers
previous_application.loc[previous_application['AMT_CREDIT'] > 6000000, 'AMT_CREDIT'] = np.nan
previous_application.loc[previous_application['SELLERPLACE_AREA'] > 3500000, 'SELLERPLACE_AREA'] = np.nan
previous_application[['DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 
'DAYS_LAST_DUE', 'DAYS_TERMINATION']].replace(365243, np.nan, inplace = True)

C:\Users\tiger\Anaconda3\lib\site-packages\pandas\core\frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [29]:
# Some new features
previous_application['prev missing'] = previous_application.isnull().sum(axis = 1).values
previous_application['prev AMT_APPLICATION / AMT_CREDIT'] = previous_application['AMT_APPLICATION'] / previous_application['AMT_CREDIT']
previous_application['prev AMT_APPLICATION - AMT_CREDIT'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_CREDIT']
previous_application['prev AMT_APPLICATION - AMT_GOODS_PRICE'] = previous_application['AMT_APPLICATION'] - previous_application['AMT_GOODS_PRICE']
previous_application['prev AMT_GOODS_PRICE - AMT_CREDIT'] = previous_application['AMT_GOODS_PRICE'] - previous_application['AMT_CREDIT']
previous_application['prev DAYS_FIRST_DRAWING - DAYS_FIRST_DUE'] = previous_application['DAYS_FIRST_DRAWING'] - previous_application['DAYS_FIRST_DUE']
previous_application['prev DAYS_TERMINATION less -500'] = (previous_application['DAYS_TERMINATION'] < -500).astype(int)

In [30]:
# Categorical features with One-Hot encode
previous_application, prev_cat = one_hot_encoder(previous_application, nan_as_category=True)

In [31]:
previous_application_agg = previous_application.groupby('SK_ID_PREV')[prev_cat].mean().reset_index()

In [32]:
previous_application_agg_new = agg_numeric(previous_application_agg.drop(columns = []), group_var = 'SK_ID_PREV', df_name = 'previous_application_agg')

In [33]:
previous_application_agg_new.shape

(1670214, 726)

In [33]:
df = df.merge(previous_application_agg_new, how='left', on='SK_ID_CURR')
del previous_application
del previous_application_agg
del previous_application_agg_new
del prev_cat
gc.collect()

77

In [34]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

Training Data Shape:  (307511, 1145)
Testing Data Shape:  (48744, 1145)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.76356	train's binary_logloss: 0.251477	valid's auc: 0.746326	valid's binary_logloss: 0.252938
[400]	train's auc: 0.783306	train's binary_logloss: 0.241989	valid's auc: 0.754784	valid's binary_logloss: 0.246867
[600]	train's auc: 0.798226	train's binary_logloss: 0.236038	valid's auc: 0.761114	valid's binary_logloss: 0.244037
[800]	train's auc: 0.810236	train's binary_logloss: 0.231391	valid's auc: 0.765307	valid's binary_logloss: 0.2424
[1000]	train's auc: 0.820717	train's binary_logloss: 0.227457	valid's auc: 0.768254	valid's binary_logloss: 0.241336
[1200]	train's auc: 0.830043	train's binary_logloss: 0.223949	valid's auc: 0.770058	valid's binary_logloss: 0.240674
[1400]	train's auc: 0.838722	train's binary_logloss: 0.220

KeyboardInterrupt: 

In [ ]:
metrics_raw

In [ ]:
df.shape

In [139]:
fi_raw.sort_values(by=['importance'], inplace=True)
drop = list(fi_raw['feature'][1:200])

In [142]:
for i in drop:
    try:
        df.drop(i, inplace=True, axis=1)
    except KeyError:
        continue

In [143]:
df.shape

(356255, 18)

In [144]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

Training Data Shape:  (307511, 137)
Testing Data Shape:  (48744, 137)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.643103	train's binary_logloss: 0.273583	valid's auc: 0.631741	valid's binary_logloss: 0.272609
[400]	train's auc: 0.649036	train's binary_logloss: 0.271257	valid's auc: 0.633587	valid's binary_logloss: 0.271062
[600]	train's auc: 0.654118	train's binary_logloss: 0.270046	valid's auc: 0.634713	valid's binary_logloss: 0.270557
[800]	train's auc: 0.658282	train's binary_logloss: 0.269229	valid's auc: 0.635202	valid's binary_logloss: 0.270379
[1000]	train's auc: 0.661659	train's binary_logloss: 0.268601	valid's auc: 0.635345	valid's binary_logloss: 0.270322
Early stopping, best iteration is:
[944]	train's auc: 0.660751	train's binary_logloss: 0.268761	valid's auc: 0.635349	valid's binary_logloss: 0.270329
[LightGBM] [Warning] nu

KeyboardInterrupt: 

In [ ]:
metrics_raw

# Credit_card_balance

In [34]:
# Replace some outliers
credit_card_balance.loc[credit_card_balance['AMT_PAYMENT_CURRENT'] > 4000000, 'AMT_PAYMENT_CURRENT'] = np.nan
credit_card_balance.loc[credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'] > 1000000, 'AMT_CREDIT_LIMIT_ACTUAL'] = np.nan

In [35]:
# Amount used from limit
credit_card_balance['LIMIT_USE'] = credit_card_balance['AMT_BALANCE'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']
# Current payment / Min payment
credit_card_balance['PAYMENT_DIV_MIN'] = credit_card_balance['AMT_PAYMENT_CURRENT'] / credit_card_balance['AMT_INST_MIN_REGULARITY']
# Late payment
credit_card_balance['LATE_PAYMENT'] = credit_card_balance['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
# How much drawing of limit
credit_card_balance['DRAWING_LIMIT_RATIO'] = credit_card_balance['AMT_DRAWINGS_ATM_CURRENT'] / credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL']

In [36]:
credit_card_balance.shape

(3840312, 27)

In [37]:
# Categorical features with One-Hot encode
credit_card_balance, categorical = one_hot_encoder(credit_card_balance, nan_as_category=True)

In [38]:
credit_card_balance_agg = credit_card_balance.groupby('SK_ID_PREV')[categorical].mean().reset_index()

In [39]:
credit_card_balance_agg.shape

(104307, 8)

In [40]:
credit_card_balance_agg = agg_numeric(credit_card_balance_agg, group_var = 'SK_ID_PREV', df_name = 'credit_card_balance_agg')
credit_card_balance_agg.head(10)

,SK_ID_PREV,credit_card_balance_agg_NAME_CONTRACT_STATUS_Active_count,credit_card_balance_agg_NAME_CONTRACT_STATUS_Active_mean,credit_card_balance_agg_NAME_CONTRACT_STATUS_Active_max,credit_card_balance_agg_NAME_CONTRACT_STATUS_Active_min,credit_card_balance_agg_NAME_CONTRACT_STATUS_Active_sum,credit_card_balance_agg_NAME_CONTRACT_STATUS_Completed_count,credit_card_balance_agg_NAME_CONTRACT_STATUS_Completed_mean,credit_card_balance_agg_NAME_CONTRACT_STATUS_Completed_max,credit_card_balance_agg_NAME_CONTRACT_STATUS_Completed_min,...,credit_card_balance_agg_NAME_CONTRACT_STATUS_Refused_count,credit_card_balance_agg_NAME_CONTRACT_STATUS_Refused_mean,credit_card_balance_agg_NAME_CONTRACT_STATUS_Refused_max,credit_card_balance_agg_NAME_CONTRACT_STATUS_Refused_min,credit_card_balance_agg_NAME_CONTRACT_STATUS_Refused_sum,credit_card_balance_agg_NAME_CONTRACT_STATUS_Approved_count,credit_card_balance_agg_NAME_CONTRACT_STATUS_Approved_mean,credit_card_balance_agg_NAME_CONTRACT_STATUS_Approved_max,credit_card_balance_agg_NAME_CONTRACT_STATUS_Approved_min,credit_card_balance_agg_NAME_CONTRACT_STATUS_Approved_sum
0,1000018,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
1,1000030,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
2,1000031,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
3,1000035,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
4,1000077,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
5,1000083,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
6,1000087,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
7,1000089,1,0.000000,0.000000,0.000000,0.000000,1,1.000000,1.000000,1.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
8,1000094,1,0.931818,0.931818,0.931818,0.931818,1,0.068182,0.068182,0.068182,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
9,1000096,1,0.687500,0.687500,0.687500,0.687500,1,0.312500,0.312500,0.312500,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0


In [41]:
credit_card_balance_agg.shape

(104307, 36)

In [42]:
previous_application_agg_new = previous_application_agg_new.merge(credit_card_balance_agg, how='left', on='SK_ID_PREV')

In [43]:
del credit_card_balance
del credit_card_balance_agg
del categorical
gc.collect()

111

In [44]:
previous_application_agg_new.shape

(1670214, 761)

In [ ]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

# POS_CASH_Balance

In [54]:
# Replace some outliers
POS_CASH_balance.loc[POS_CASH_balance['CNT_INSTALMENT_FUTURE'] > 60, 'CNT_INSTALMENT_FUTURE'] = np.nan

In [55]:
# Some new features
POS_CASH_balance['pos CNT_INSTALMENT more CNT_INSTALMENT_FUTURE'] = \
                    (POS_CASH_balance['CNT_INSTALMENT'] > POS_CASH_balance['CNT_INSTALMENT_FUTURE']).astype(int)

In [56]:
POS_CASH_balance, categorical = one_hot_encoder(POS_CASH_balance, nan_as_category=True)

In [57]:
POS_CASH_balance_agg = POS_CASH_balance.groupby('SK_ID_PREV')[categorical].mean().reset_index()

In [58]:
POS_agg = agg_numeric(POS_CASH_balance_agg, group_var = 'SK_ID_PREV', df_name = 'POS_CASH_balance_agg')
POS_agg.head(10)

,SK_ID_PREV,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Active_count,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Active_mean,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Active_max,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Active_min,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Active_sum,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Completed_count,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Completed_mean,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Completed_max,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Completed_min,...,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_XNA_count,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_XNA_mean,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_XNA_max,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_XNA_min,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_XNA_sum,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Amortized debt_count,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Amortized debt_mean,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Amortized debt_max,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Amortized debt_min,POS_CASH_balance_agg_NAME_CONTRACT_STATUS_Amortized debt_sum
0,1000001,1,0.666667,0.666667,0.666667,0.666667,1,0.333333,0.333333,0.333333,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
1,1000002,1,0.800000,0.800000,0.800000,0.800000,1,0.200000,0.200000,0.200000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
2,1000003,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
3,1000004,1,0.875000,0.875000,0.875000,0.875000,1,0.125000,0.125000,0.125000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
4,1000005,1,0.909091,0.909091,0.909091,0.909091,1,0.090909,0.090909,0.090909,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
5,1000007,1,1.000000,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
6,1000008,1,0.900000,0.900000,0.900000,0.900000,1,0.100000,0.100000,0.100000,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
7,1000009,1,0.857143,0.857143,0.857143,0.857143,1,0.142857,0.142857,0.142857,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
8,1000010,1,0.909091,0.909091,0.909091,0.909091,1,0.090909,0.090909,0.090909,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0
9,1000011,1,0.923077,0.923077,0.923077,0.923077,1,0.076923,0.076923,0.076923,...,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0


In [60]:
POS_agg.shape

(936325, 46)

In [61]:
previous_application_agg_new = previous_application_agg_new.merge(POS_agg, how='left', on='SK_ID_PREV')

In [62]:
previous_application_agg_new.shape

(1670214, 861)

# Instalments payments

In [45]:
# Replace some outliers
installments_payments.loc[installments_payments['NUM_INSTALMENT_VERSION'] > 70, 'NUM_INSTALMENT_VERSION'] = np.nan
installments_payments.loc[installments_payments['DAYS_ENTRY_PAYMENT'] < -4000, 'DAYS_ENTRY_PAYMENT'] = np.nan

In [46]:
# Some new features
installments_payments['ins DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
installments_payments['ins NUM_INSTALMENT_NUMBER_100'] = (installments_payments['NUM_INSTALMENT_NUMBER'] == 100).astype(int)
installments_payments['ins DAYS_INSTALMENT more NUM_INSTALMENT_NUMBER'] = (installments_payments['DAYS_INSTALMENT'] > installments_payments['NUM_INSTALMENT_NUMBER'] * 50 / 3 - 11500 / 3).astype(int)
installments_payments['ins AMT_INSTALMENT - AMT_PAYMENT'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']
installments_payments['ins AMT_PAYMENT / AMT_INSTALMENT'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT']

In [47]:
# Categorical features with One-Hot encode
installments_payments, categorical = one_hot_encoder(installments_payments, nan_as_category=True)

In [49]:
installments_payments_agg = agg_numeric(installments_payments, group_var = 'SK_ID_PREV', df_name = 'installments_payments')

In [50]:
installments_payments_agg.shape

(997752, 56)

In [51]:
previous_application_agg_new = previous_application_agg_new.merge(installments_payments_agg, how='left', on='SK_ID_PREV')

In [52]:
previous_application_agg_new.shape

(1670214, 816)

In [53]:
del installments_payments
del installments_payments_agg
del categorical
gc.collect()

89

# Aggregate Previous application

In [66]:
installments_payments = pd.read_csv(r"C:\Users\tiger\Desktop\Home Credit Default Risk\installments_payments.csv")

In [83]:
ids = installments_payments.iloc[:,0:2]

In [ ]:
previous_application_agg_new = previous_application_agg_new.merge(ids, how='left', on='SK_ID_PREV')

In [ ]:
previous_application_agg_new.shape

In [ ]:
previous_application_agg_new = agg_numeric(previous_application_agg_new group_var = 'SK_ID_CURR', df_name = 'previous_application_agg_new')

In [ ]:
previous_application_agg_new.shape

In [ ]:
# group and merge df and previous_application_agg_new

df = df.merge(previous_application_agg_new, how='left', on='SK_ID_CURR')

In [ ]:
del previous_application_agg_new
del installments_payments
gc.collect()

In [ ]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]
submission_raw, fi_raw, metrics_raw = model(train_df, test_df)

In [ ]:
metrics_raw

In [ ]:
plot_feature_importances(fi_raw)

In [26]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # LIGHTGBM CONFIGURATION AND HYPER-PARAMETERS
    GENERATE_SUBMISSION_FILES = True
    STRATIFIED_KFOLD = False
    RANDOM_SEED = 50
    NUM_FOLDS = 10
    EARLY_STOPPING = 100
    NUM_THREADS = 4

    LIGHTGBM_PARAMS = {
        'boosting_type': 'goss',
        'n_estimators': 10000,
        'learning_rate': 0.005134,
        'num_leaves': 54,
        'max_depth': 10,
        'subsample_for_bin': 240000,
        'reg_alpha': 0.436193,
        'reg_lambda': 0.479169,
        'colsample_bytree': 0.508716,
        'min_split_gain': 0.024766,
        'subsample': 1,
        'is_unbalance': False,
        'silent':-1,
        'verbose':-1
    }
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        params = {'random_state': RANDOM_SEED, 'nthread': NUM_THREADS}
        model = LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [27]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance'].head(15), 
            align = 'center', edgecolor = 'g')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Feature Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df